## Connecting to a Mongo Database and  Twitter via the Twitter API. 

In [1]:
import os
os.chdir('../')

In [2]:
from lib.pull_tweets import connect_to_mongo, get_twitter_client, select_all_tweets, get_user_tweets
import twitter, yaml, tweepy
import pymongo
import pandas as pd
from pymongo import MongoClient
import csv
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from unidecode import unidecode
from lib.cleaning_module import clean_text
from nltk.tokenize import word_tokenize
import pickle
from tweepy import OAuthHandler
import json

In [3]:
client_aws = connect_to_mongo()

[u'creator_tweets', u'sage_tweets', u'tweets', u'elon', u'russell', u'madonna', u'magician_tweets', u'jester_tweets', u'seth', u'paolo', u'pers', u'katy', u'explorer_tweets', u'barack', u'lover_tweets', u'bill', u'zooey', u'hero_tweets', u'kid', u'oz', u'ellen', u'innocent_tweets', u'gina', u'rebel_tweets', u'jessica', u'donald', u'ruler_tweets', u'michael', u'hillary', u'noam', u'nurturer_tweets', u'oprah', u'pope', u'bourdain', u'martha', u'sully', u'kim', u'every_tweets']


In [4]:
access_key =  ''
access_secret = ''
consumer_token =  ''
consumer_secret = ''

## Getting one twitter history feed per archetype

In [5]:
def get_user_tweets(screen_name):

    auth = OAuthHandler(consumer_token, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    alltweets = []    
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    alltweets.extend(new_tweets)
    user=api.get_user(screen_name=screen_name)
    oldest = alltweets[-1].id - 1
    
    while len(new_tweets) > 0:
        new_tweets = api.user_timeline(screen_name = screen_name, count=200,max_id=oldest)
        alltweets.extend(new_tweets)
        oldest = alltweets[-1].id - 1
    return alltweets

In [ ]:
all_zooey = get_user_tweets('ZooeyDeschanel')
for tweet in all_zooey:
    client_aws.zooey.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_kid = get_user_tweets('iamkidpresident')
for tweet in all_kid:
    client_aws.kid.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_oz = get_user_tweets('DrOz')
for tweet in all_oz:
    client_aws.oz.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_jessica = get_user_tweets('jessicaalba')
for tweet in all_jessica:
    client_aws.jessica.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_martha = get_user_tweets('MarthaStewart')
for tweet in all_martha:
    client_aws.martha.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_sully = get_user_tweets('Captsully')
for tweet in all_sully:
    client_aws.sully.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_bill = get_user_tweets('billmaher')
for tweet in all_bill:
    client_aws.bill.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_madonna = get_user_tweets('Madonna')
for tweet in all_madonna:
    client_aws.madonna.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_seth = get_user_tweets('SethMacFarlane')
for tweet in all_seth:
    client_aws.seth.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_pope = get_user_tweets('Pontifex')
for tweet in all_pope:
    client_aws.pope.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_hillary = get_user_tweets('HillaryClinton')
for tweet in all_hillary:
    client_aws.hillary.insert_one(json.loads(json.dumps(tweet._json)))

In [ ]:
all_anthony = get_user_tweets('bourdain')
for tweet in all_anthony:
    client_aws.anthony.insert_one(json.loads(json.dumps(tweet._json)))

In [6]:
def get_texts_and_user(user):
    tweets = get_user_tweets(user)
    return pd.concat([pd.DataFrame([[i.user.screen_name, i.text]], columns=['user','text']) for i in tweets],axis=0).reset_index(drop=True)

In [7]:
jessica_df = get_texts_and_user('jessicaalba')
louis_df = get_texts_and_user('Louis_Tomlinson')
joseph_df = get_texts_and_user('hitRECordJoe')
zooey_df = get_texts_and_user('ZooeyDeschanel')
channing_df = get_texts_and_user('channingtatum')
jamie_df = get_texts_and_user('iamjamiefoxx')
gavin_df = get_texts_and_user('GavinNewsom')
justin_df = get_texts_and_user('JustinTrudeau')
sully_df = get_texts_and_user('Captsully')
reese_df = get_texts_and_user('RWitherspoon')
deepak_df = get_texts_and_user('DeepakChopra')
kid_df = get_texts_and_user('iamkidpresident')
mark_df = get_texts_and_user('mcuban')
martha_df = get_texts_and_user('MarthaStewart')
shonda_df = get_texts_and_user('shondarhimes')
laverne_df = get_texts_and_user('Lavernecox')
christiane_df = get_texts_and_user('camanpour')
courtney_df = get_texts_and_user('Courtney')
bill_df = get_texts_and_user('billmaher')
lena_df = get_texts_and_user('lenadunham')
madonna_df = get_texts_and_user('Madonna')
usher_df = get_texts_and_user('Usher')
weeknd_df = get_texts_and_user('theweeknd')
james_df = get_texts_and_user('JKCorden')
seth_df = get_texts_and_user('SethMacFarlane')
sarah_df = get_texts_and_user('SarahKSilverman')
dalai_df = get_texts_and_user('dalailama')
pope_df = get_texts_and_user('Pontifex')
diane_df = get_texts_and_user('DianeSawyer')
oz_df = get_texts_and_user('DrOz')
tony_df = get_texts_and_user('TonyRobbins')
lori_df = get_texts_and_user('LoriGreiner')
david_df = get_texts_and_user('David_Cameron')
marissa_df = get_texts_and_user('marissamayer')
hillary_df = get_texts_and_user('HillaryClinton')
gina_df = get_texts_and_user('HereIsGina')
katy_df = get_texts_and_user('katyperry')
barack_df = get_texts_and_user('BarackObama')
oprah_df = get_texts_and_user('Oprah')
noam_df = get_texts_and_user('noamchomskyT')
michael_df = get_texts_and_user('MMFlint')
kim_df = get_texts_and_user('KimKardashian')
russell_df = get_texts_and_user('rustyrockets')
ellen_df = get_texts_and_user('TheEllenShow')
paulo_df = get_texts_and_user('paulocoelho')
elon_df = get_texts_and_user('elonmusk')
donald_df = get_texts_and_user('realDonaldTrump')
anthony_df = get_texts_and_user('bourdain')
emma_df = get_texts_and_user('EmmaWatson')
iyanla_df = get_texts_and_user('IyanlaVanzant')

In [8]:
all_tweets2 = [gina_df, katy_df, barack_df, oprah_df, noam_df, michael_df, kim_df, russell_df, ellen_df, paulo_df, elon_df, donald_df]
all_tweets2_df = pd.concat(all_tweets2)

In [ ]:
all_tweets_alt = [jessica_df, zooey_df, sully_df, kid_df, anthony_df, bill_df, madonna_df, martha_df, seth_df, pope_df, oz_df, hillary_df]
all_tweets_alt_df = pd.concat(all_tweets_alt)

## Pickling the full dataset of all tweets

In [ ]:
with open('all_tweets_alt_df.pkl', 'w') as picklefile:
    pickle.dump(all_tweets_alt_df, picklefile)

In [9]:
with open('all_tweets2_df.pkl', 'w') as picklefile:
    pickle.dump(all_tweets2_df, picklefile)